In [23]:
import os
import tempfile

import gensim.corpora.mmcorpus as gm
import gensim.models.ldamodel as gl
import gensim.similarities as gs

import requests

In [16]:
models_uri = 'https://paperscopestorage.blob.core.windows.net/models'

In [30]:
mm_fname = 'theringer.mm'
mm_urls = [f'{models_uri}/{mm_fname}{suffix}' for suffix in ('', '.index')]
mm_tmpdir = tempfile.TemporaryDirectory()
for url in mm_urls:
    fname = url.rsplit('/', 1)[1]
    res = requests.get(url)
    with open(os.path.join(mm_tmpdir.name, fname), 'wb') as fh:
        fh.write(res.content)

corpus = gm.MmCorpus(os.path.join(mm_tmpdir.name, mm_fname))

In [39]:
lda_fname = 'theringer.lda'
lda_urls = [
    f'{models_uri}/{lda_fname}{suffix}'
    for suffix in ('', '.expElogbeta.npy', '.id2word', '.state')
]
lda_tmpdir = tempfile.TemporaryDirectory()
for url in lda_urls:
    fname = url.rsplit('/', 1)[1]
    res = requests.get(url)
    with open(os.path.join(lda_tmpdir.name, fname), 'wb') as fh:
        fh.write(res.content)

lda = gl.LdaModel.load(os.path.join(lda_tmpdir.name, lda_fname))

In [32]:
sim_fname = 'theringer.index'
sim_urls = [f'{models_uri}/{sim_fname}{suffix}' for suffix in ('', '.0')]
sim_tmpdir = tempfile.TemporaryDirectory()
for url in sim_urls:
    fname = url.rsplit('/', 1)[1]
    res = requests.get(url)
    with open(os.path.join(sim_tmpdir.name, fname), 'wb') as fh:
        fh.write(res.content)

sim = gs.SparseMatrixSimilarity.load(os.path.join(sim_tmpdir.name, sim_fname))

In [36]:
for shard in sim.shards:
    shard.dirname = sim_tmpdir.name

In [41]:
sim.num_best = 3
sim[lda[corpus[102]]]

[(102, 0.9171121120452881),
 (3584, 0.9170953631401062),
 (119, 0.91709303855896)]